#### order brushing
- 先找出 deemed shop
- 再找出該 shop 下的 deemed buyers


In [40]:
import pandas as pd
from datetime import datetime
from tqdm import tqdm
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"


In [104]:
def gen_dayHrMin(user_clicks, time_col):
    # # 將 日-時-分 拆出來
#     user_clicks['year'] = user_clicks[time_col].apply(lambda x: int(x.split(' ')[0].split('-')[0]))
#     user_clicks['month'] = user_clicks[time_col].apply(lambda x: int(x.split(' ')[0].split('-')[1]))
    user_clicks['day'] = user_clicks[time_col].apply(lambda x: int(x.split(' ')[0].split('-')[2]))
    user_clicks['hour'] = user_clicks[time_col].apply(lambda x: int(x.split(' ')[1].split(':')[0]))
    user_clicks['minute'] = user_clicks[time_col].apply(lambda x: int(x.split(' ')[1].split(':')[1]))
    user_clicks['second'] = user_clicks[time_col].apply(lambda x: int(x.split(' ')[1].split(':')[2]))
#     user_clicks.drop(time_col, axis=1, inplace=True)
    return user_clicks

def gen_timeblock(df):
    df['timeblock'] = (df['day'] - 27) * 24 + df['hour']
    df['min_block'] = (df['day'] - 27) * 24 * 60 + df['hour'] * 60 + df['minute']
    df['sec_block'] = (df['day'] - 27) * 24 * 3600 + df['hour'] * 3600 + df['minute'] * 60 + df['second']
    df = df.sort_values('sec_block')
    return df


In [105]:
df = pd.read_csv('order_brush_order.csv')
df.isnull().sum()
len(df)
df['orderid'] = df['orderid'].astype(str)
df['shopid'] = df['shopid'].astype(str)
df['userid'] = df['userid'].astype(str)
df = gen_dayHrMin(df, 'event_time')
df = gen_timeblock(df)
df.head()


orderid       0
shopid        0
userid        0
event_time    0
dtype: int64

222750

,orderid,shopid,userid,event_time,day,hour,minute,second,timeblock,min_block,sec_block
150060,31075200506751,6042309,97707522,2019-12-27 00:00:00,27,0,0,0,0,0,0
25985,31075200506752,104804492,97707522,2019-12-27 00:00:00,27,0,0,0,0,0,0
81020,31075200506753,8715449,97707522,2019-12-27 00:00:00,27,0,0,0,0,0,0
146598,31075201870570,190969466,170182475,2019-12-27 00:00:02,27,0,0,2,0,0,2
80355,31075205798264,2859407,12532131,2019-12-27 00:00:05,27,0,0,5,0,0,5


In [ ]:
deemed_records_2 = list()
for shop in tqdm(df.shopid.unique()):
    tmp = df[df['shopid']==shop].sort_values('sec_block')
    for sec in tmp.sec_block.unique():
        concentrate_rate = len(tmp[(tmp['sec_block']>=sec)&(tmp['sec_block']<=sec+3600)].orderid.values) / len(tmp[(tmp['sec_block']>=sec)&(tmp['sec_block']<=sec+3600)].userid.unique())
        if concentrate_rate >= 3:
            print(shop, sec, sec+3600, concentrate_rate)
            tmp[(tmp['sec_block']>=sec)&(tmp['sec_block']<=sec+3600)]
            protion_df = tmp[(tmp['sec_block']>=sec)&(tmp['sec_block']<=sec+3600)].groupby(['userid'])['orderid'].count().reset_index()
            protion_df['portion'] = protion_df['orderid'] / protion_df['orderid'].sum()
            protion_df
            max(protion_df['portion'].values)
            deemed_buyers = protion_df[protion_df['portion']==max(protion_df['portion'].values)].userid.unique()
            deemed_buyers
            for deemed_buyer in deemed_buyers:
                deemed_records_2.append([shop, deemed_buyer])


In [123]:
df_result_2 = pd.DataFrame(deemed_records_2)
df_result_2.columns = ['shopid','userid']
df_result_2.drop_duplicates(inplace=True)
df_result_2 = df_result_2.groupby('shopid').agg({'userid': '&'.join}).reset_index()
df_result_2


,shopid,userid
0,100446829,2434757
1,10199219,8405753
2,10206302,95058664
3,103715156,214226569
4,10402,77819
...,...,...
271,97167904,214778616
272,98481320,124597967
273,98793086,142710562
274,99067259,108402614


In [114]:
# output deemed_records
deemed_records = list()
for shop in tqdm(df.shopid.unique()):
    tmp = df[df['shopid']==shop].sort_values('event_time')
    for block in tmp.timeblock.unique():
        tmp = tmp[tmp['timeblock']==block]
        if len(tmp) > 0:
            concentrate_rate = len(tmp.orderid.values) / len(tmp.userid.unique())
            if concentrate_rate >= 3:
                print(shop, block, concentrate_rate)
                protion_df = tmp.groupby(['userid'])['orderid'].count().reset_index()
                protion_df['portion'] = protion_df['orderid'] / protion_df['orderid'].sum()
                deemed_buyers = protion_df[protion_df['portion']==max(protion_df['portion'].values)].userid.unique()
                for deemed_buyer in deemed_buyers:
                    deemed_records.append([shop, deemed_buyer])
        
        tmp = tmp[(tmp['timeblock']==block)|(tmp['timeblock']==block+1)]
        if len(tmp) > 0:
            concentrate_rate = len(tmp.orderid.values) / len(tmp.userid.unique())
            if concentrate_rate >= 3:
                print(shop, block, block+1, concentrate_rate)
                protion_df = tmp.groupby(['userid'])['orderid'].count().reset_index()
                protion_df['portion'] = protion_df['orderid'] / protion_df['orderid'].sum()
                deemed_buyers = protion_df[protion_df['portion']==max(protion_df['portion'].values)].userid.unique()
                for deemed_buyer in deemed_buyers:
                    deemed_records.append([shop, deemed_buyer])









  0%|                                                                                        | 0/18770 [00:00<?, ?it/s]






  0%|                                                                                | 3/18770 [00:00<22:22, 13.98it/s]






  0%|                                                                                | 7/18770 [00:00<18:51, 16.58it/s]






  0%|                                                                                | 9/18770 [00:00<28:31, 10.96it/s]






  0%|                                                                               | 11/18770 [00:00<27:43, 11.28it/s]






  0%|                                                                               | 13/18770 [00:01<27:23, 11.41it/s]






  0%|                                                                               | 15/18770 [00:01<25:39, 12.18it/s]






  0%|                                                                               | 17/18770 [00:01<28:43, 10

  1%|▋                                                                             | 159/18770 [00:10<16:25, 18.89it/s]






  1%|▋                                                                             | 162/18770 [00:10<19:32, 15.87it/s]






  1%|▋                                                                             | 164/18770 [00:10<21:02, 14.73it/s]






  1%|▋                                                                             | 167/18770 [00:11<19:03, 16.27it/s]






  1%|▋                                                                             | 170/18770 [00:11<20:42, 14.97it/s]






  1%|▋                                                                             | 172/18770 [00:11<23:57, 12.94it/s]






  1%|▋                                                                             | 174/18770 [00:11<26:55, 11.51it/s]






  1%|▋                                                                             | 176/18770 [00:11<23:30, 13.18it/s

  3%|██                                                                            | 483/18770 [00:30<16:18, 18.69it/s]






  3%|██                                                                            | 486/18770 [00:30<14:46, 20.62it/s]






  3%|██                                                                            | 489/18770 [00:31<14:27, 21.06it/s]






  3%|██                                                                            | 492/18770 [00:31<16:15, 18.73it/s]






  3%|██                                                                            | 494/18770 [00:31<16:30, 18.44it/s]






  3%|██                                                                            | 497/18770 [00:31<15:00, 20.29it/s]






  3%|██                                                                            | 500/18770 [00:31<16:58, 17.94it/s]






  3%|██                                                                            | 502/18770 [00:31<17:22, 17.52it/s

181009364 0 3.0
181009364 0 1 3.0









  4%|███▏                                                                          | 768/18770 [00:46<18:06, 16.56it/s]






  4%|███▏                                                                          | 771/18770 [00:46<17:29, 17.15it/s]






  4%|███▏                                                                          | 773/18770 [00:47<20:10, 14.86it/s]






  4%|███▏                                                                          | 775/18770 [00:47<22:47, 13.16it/s]






  4%|███▏                                                                          | 777/18770 [00:47<24:50, 12.08it/s]






  4%|███▏                                                                          | 779/18770 [00:47<25:05, 11.95it/s]






  4%|███▏                                                                          | 781/18770 [00:47<23:20, 12.85it/s]






  4%|███▎                                                                          | 783/18770 [00:47<23:54, 12

...

 98%|██████████████████████████████████████████████████████████████████████████▏ | 18308/18770 [09:11<00:10, 44.65it/s]






 98%|██████████████████████████████████████████████████████████████████████████▏ | 18313/18770 [09:11<00:10, 44.85it/s]






 98%|██████████████████████████████████████████████████████████████████████████▏ | 18318/18770 [09:11<00:09, 45.24it/s]






 98%|██████████████████████████████████████████████████████████████████████████▏ | 18323/18770 [09:11<00:09, 45.02it/s]






 98%|██████████████████████████████████████████████████████████████████████████▏ | 18328/18770 [09:11<00:09, 44.75it/s]






 98%|██████████████████████████████████████████████████████████████████████████▏ | 18333/18770 [09:11<00:09, 44.56it/s]






 98%|██████████████████████████████████████████████████████████████████████████▎ | 18338/18770 [09:12<00:09, 44.85it/s]






 98%|██████████████████████████████████████████████████████████████████████████▎ | 18343/18770 [09:12<00:09, 45.00it/s

123401085 115 4.0
123401085 115 116 4.0









 98%|██████████████████████████████████████████████████████████████████████████▌ | 18413/18770 [09:13<00:08, 43.82it/s]






 98%|██████████████████████████████████████████████████████████████████████████▌ | 18418/18770 [09:13<00:07, 44.39it/s]






 98%|██████████████████████████████████████████████████████████████████████████▌ | 18423/18770 [09:14<00:07, 44.25it/s]






 98%|██████████████████████████████████████████████████████████████████████████▌ | 18428/18770 [09:14<00:07, 44.16it/s]






 98%|██████████████████████████████████████████████████████████████████████████▋ | 18433/18770 [09:14<00:07, 44.32it/s]






 98%|██████████████████████████████████████████████████████████████████████████▋ | 18438/18770 [09:14<00:07, 45.23it/s]






 98%|██████████████████████████████████████████████████████████████████████████▋ | 18443/18770 [09:14<00:07, 45.19it/s]






 98%|██████████████████████████████████████████████████████████████████████████▋ | 18448/18770 [09:14<00:07, 45

155016230 117 3.0
155016230 117 118 3.0









 99%|███████████████████████████████████████████████████████████████████████████ | 18548/18770 [09:16<00:05, 42.20it/s]






 99%|███████████████████████████████████████████████████████████████████████████ | 18553/18770 [09:16<00:05, 43.26it/s]






 99%|███████████████████████████████████████████████████████████████████████████▏| 18558/18770 [09:17<00:04, 43.92it/s]






 99%|███████████████████████████████████████████████████████████████████████████▏| 18563/18770 [09:17<00:04, 44.16it/s]






 99%|███████████████████████████████████████████████████████████████████████████▏| 18568/18770 [09:17<00:04, 44.20it/s]






 99%|███████████████████████████████████████████████████████████████████████████▏| 18573/18770 [09:17<00:04, 44.65it/s]






 99%|███████████████████████████████████████████████████████████████████████████▏| 18578/18770 [09:17<00:04, 44.73it/s]






 99%|███████████████████████████████████████████████████████████████████████████▏| 18583/18770 [09:17<00:04, 45

In [115]:
df_result = pd.DataFrame(deemed_records)
df_result.columns = ['shopid','userid']
df_result['shopid'] = df_result['shopid'].astype(str)
df_result['userid'] = df_result['userid'].astype(str)
df_result.drop_duplicates(inplace=True)
df_result = df_result.groupby('shopid').agg({'userid': '&'.join}).reset_index()
df_result


,shopid,userid
0,100446829,2434757
1,10536,672345
2,106051591,1276208
3,123401085,12869645
4,129113839,215408773
5,132583615,152710183
6,141025402,613919
7,144365607,214411855
8,153444897,169948888
9,155016230,188431260


In [124]:
test = df_result_2.copy()
for s in tqdm(df.shopid.unique()):
    if s not in df_result_2.shopid.unique():
        test = test.append(pd.DataFrame([[s,0]],columns=['shopid','userid']), ignore_index=True)
        
test








  0%|                                                                                        | 0/18770 [00:00<?, ?it/s]






  0%|▎                                                                             | 90/18770 [00:00<00:21, 888.79it/s]






  1%|▋                                                                            | 176/18770 [00:00<00:21, 877.87it/s]






  1%|█                                                                            | 258/18770 [00:00<00:21, 857.61it/s]






  2%|█▍                                                                           | 344/18770 [00:00<00:21, 857.64it/s]






  2%|█▊                                                                           | 430/18770 [00:00<00:21, 857.66it/s]






  3%|██                                                                           | 508/18770 [00:00<00:21, 830.74it/s]






  3%|██▍                                                                          | 589/18770 [00:00<00:22, 823

 27%|████████████████████▏                                                       | 4989/18770 [00:06<00:18, 726.06it/s]






 27%|████████████████████▌                                                       | 5063/18770 [00:06<00:18, 728.47it/s]






 27%|████████████████████▊                                                       | 5136/18770 [00:06<00:18, 721.90it/s]






 28%|█████████████████████                                                       | 5209/18770 [00:06<00:18, 717.26it/s]






 28%|█████████████████████▍                                                      | 5282/18770 [00:07<00:18, 719.46it/s]






 29%|█████████████████████▋                                                      | 5354/18770 [00:07<00:18, 718.99it/s]






 29%|█████████████████████▉                                                      | 5426/18770 [00:07<00:18, 715.50it/s]






 29%|██████████████████████▎                                                     | 5498/18770 [00:07<00:18, 714.14it/s

 72%|██████████████████████████████████████████████████████▏                    | 13547/18770 [00:19<00:09, 576.78it/s]






 72%|██████████████████████████████████████████████████████▎                    | 13606/18770 [00:20<00:08, 579.29it/s]






 73%|██████████████████████████████████████████████████████▌                    | 13665/18770 [00:20<00:08, 580.28it/s]






 73%|██████████████████████████████████████████████████████▊                    | 13727/18770 [00:20<00:08, 589.47it/s]






 73%|███████████████████████████████████████████████████████                    | 13787/18770 [00:20<00:08, 586.04it/s]






 74%|███████████████████████████████████████████████████████▎                   | 13846/18770 [00:20<00:08, 586.73it/s]






 74%|███████████████████████████████████████████████████████▌                   | 13905/18770 [00:20<00:08, 585.38it/s]






 74%|███████████████████████████████████████████████████████▊                   | 13964/18770 [00:20<00:08, 583.68it/s

,shopid,userid
0,100446829,2434757
1,10199219,8405753
2,10206302,95058664
3,103715156,214226569
4,10402,77819
...,...,...
18765,173249173,0
18766,212058195,0
18767,577768,0
18768,162561288,0


In [52]:
df_result.to_csv('partial_result.csv')

In [126]:
test.to_csv('200613_6th_submission.csv', index=0)

In [97]:
test['userid'] = test['userid'].astype(str)
test[test['userid'].str.contains('&')]

,shopid,userid
60,136564914,191211430&178491887
120,175531295,187697407&215009429
124,181009364,214208720&101832161
149,201503467,130587573&92521144
190,51134277,29857724&212200633
191,51487211,5307816&214808165
193,54257623,1974334&107414154
243,8996761,162508227&13135622&215382704
